# Sentiment Analysis using RNN

### Load in and visualize the data

In [1]:
import numpy as np
with open('data/reviews.txt','r') as f:
    reviews = f.read()
with open('data/labels.txt','r') as f:
    labels = f.read()

In [2]:
print(reviews[:2000])
print(labels[:24])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Data Pre-processing

Steps:
- Get rid of periods and extraneous punctuation
- Split the text into each review using '\n' as delimiter
- Combine all the reviews back together into one big string

In [3]:
#Removing the punctuation
from string import punctuation

print(punctuation)

reviews = reviews.lower()    #Standardizing the text as lowercase
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
#Split by new-lines and spaces
reviews_split = all_text.split("\n")
all_text = ' '.join(reviews_split)

#Create a list of words
words = all_text.split()

In [5]:
print(words[:30])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me']


### Encoding the words

In [6]:
# Creating a dictionary to encode words in numbers starting from 1
from collections import Counter

#Building a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts,key=counts.get,reverse=True)
vocab_to_int = {word:ii for ii,word in enumerate(vocab,1)}

#Using the dict to tokenize each review in reviews_split
reviews_ints = []
for r in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in r.split()])

In [7]:
#Testing the above code

#Stats about vocabulary
print("Unique words:",len(vocab_to_int))

#Print tokens in first review
print("\nTokenized Review: \n",reviews_ints[:1])

Unique words: 74072

Tokenized Review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


### Encoding the labels

In [8]:
#Encoding positive as 1 and negative as 0
encoded_labels = np.array([1 if label=="positive" else 0 for label in labels.split("\n")])
# print(len(encoded_labels))
print(encoded_labels[:3])

[1 0 1]


### Removing outliers
Our network will expect a standard input text size, and hence we'll shape our reviews into a specific length. The approach taken is as follows:
- Getting rid of extremely long or short reviews - The outliers
- Padding/Truncating the remaining data so that we have reviews of the same length

In [9]:
#Outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum length review: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum length review: 2514


In [10]:
#Removing zero-length reviews
print("No. of reviews before removing outliers: {}".format(len(reviews_ints)))

non_zero_idx = [ii for ii,review in enumerate(reviews_ints) if len(review) != 0]

reviews_ints = [ reviews_ints[ii] for ii in non_zero_idx ]
encoded_labels = [ encoded_labels[ii] for ii in non_zero_idx ]

print("No. of reviews after removing outliers: {}".format(len(reviews_ints)))

No. of reviews before removing outliers: 25001
No. of reviews after removing outliers: 25000


### Padding sequences
To deal with both short and long reviews, we'll pad or truncate all our reviews. For reviews shorter than some seq_length, we'll pad with 0s. For reviews longer than seq_length, we can truncate them to the first seq_length words.

In [11]:
def pad_features(reviews_ints, seq_length):
    '''
    Returns the features of reviews_ints, where each review is padded with 0's or 
    truncated to input seq_length
    '''
    features = []
    for review in reviews_ints:
        if len(review) < seq_length:
            features.append([0]*(seq_length-len(review))+review)
        else:
            features.append(review[:seq_length])
    return np.array(features)

In [12]:
#Testing implementation of above function

seq_length = 200
features = pad_features(reviews_ints,seq_length)

assert len(features) == len(reviews_ints)    #Features should have same number of rows as reviews_ints
assert len(features[0]) == seq_length        #Each feature row should contain 'seq_length' values

print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

## Training, Validation, Test

In [13]:
split_frac = 0.8

#Splitting data into training, validation and test sets
split_idx = int(len(features)*split_frac)

#Training set
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

#Validation and test sets
remaining_split_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:remaining_split_idx], remaining_x[remaining_split_idx:]
val_y, test_y = remaining_y[:remaining_split_idx], remaining_y[remaining_split_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


## Dataloaders and Batching the data

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

#Creating tensor datasets
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))
val_data = TensorDataset(torch.from_numpy(np.array(val_x)), torch.from_numpy(np.array(val_y)))

batch_size = 50

#Dataloaders
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [15]:
#Obtain one batch of training data

dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[  54,    8,  379,  ...,   37, 2559,   26],
        [1281,   10,  329,  ...,  447,    4,    1],
        [   0,    0,    0,  ...,  825,    6,  470],
        ...,
        [  11,    6,    3,  ..., 1939,   26,  551],
        [   0,    0,    0,  ...,   39,   11,   20],
        [   0,    0,    0,  ...,    5, 2260,   56]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 0], dtype=torch.int32)


## Defining the model

We will define the model with the following layers:
- An **embedding layer** that converts our work tokens (integers) into embeddings of a specific size.
- An **LSTM layer** defined by a hidden_state size and number of layers
- A **fully-connected output layer** that maps the output of LSTM layer outputs to a desired output size
- A **sigmoid activation layer** which turns all outputs into a value 0-1.

In [16]:
#Checking if GPU is available or not
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("GPU Available..Training on GPU !")
else:
    print("GPU not available.. Training on CPU !")

GPU Available..Training on GPU !


In [17]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """The RNN Model that will be used to perform Sentiment Analysis"""
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob = 0.5):
        """Initialize the model by setting up the layers"""
        super(SentimentRNN,self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        #Defining the layers
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, self.n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim,output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, hidden):
        """Perform a forward pass of our model on some input and hidden state"""
        
        batch_size = x.size(0)
        
        
        #Embeddings and LSTM Out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        #Stacking up LSTM Outputs
        lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
        
        #Dropout and Fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        #Sigmoid output
        sig_out = self.sigmoid(out)
        
        #Reshaping to be batch-size first
        sig_out = sig_out.view(batch_size,-1)
        sig_out = sig_out[:,-1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers,batch_size,self.hidden_dim).zero_(),
                      weight.new(self.n_layers,batch_size,self.hidden_dim).zero_())
            
        return hidden

### Instantiating the network

In [18]:
#Instantiate the hyperparameters
vocab_size = len(vocab_to_int) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size=vocab_size, output_size=output_size, embedding_dim=embedding_dim, 
                   hidden_dim=hidden_dim, n_layers=n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


## Training the network

In [19]:
#Loss and optimization functions

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.001)

torch.backends.cudnn.enabled = False

In [20]:
#Training params

epochs = 4
counter = 0
print_every = 100
clip = 5  #for gradient clipping

if train_on_gpu:
    net.cuda()

net.train()
#train for some number of epochs
for e in range(epochs):
    #initialize hidden state
    h = net.init_hidden(batch_size)
    
    #batch-loop
    for inputs, labels in train_loader:
        counter += 1
        
        if train_on_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        #Creating new variables for hidden state 
        h = tuple([each.data for each in h])
        
        net.zero_grad()
        
        #Get output from the model
        output, h = net(inputs, h)
        
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            #get validation loss
            val_h = net.init_hidden(batch_size)
            net.eval()
            val_losses = []
            for inputs, labels in val_loader:
                val_h = tuple([each.data for each in val_h])
                
                if train_on_gpu:
                    inputs, labels = inputs.cuda(), labels.cuda()
                    
                output_val, val_h = net(inputs, val_h)
                val_loss = criterion(output_val.squeeze(), labels.float())
                
                val_losses.append(val_loss.item())
            
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))        

Epoch: 1/4... Step: 100... Loss: 0.687573... Val Loss: 0.640764
Epoch: 1/4... Step: 200... Loss: 0.693543... Val Loss: 0.678182
Epoch: 1/4... Step: 300... Loss: 0.584024... Val Loss: 0.600235
Epoch: 1/4... Step: 400... Loss: 0.648290... Val Loss: 0.574703
Epoch: 2/4... Step: 500... Loss: 0.644994... Val Loss: 0.626900
Epoch: 2/4... Step: 600... Loss: 0.547131... Val Loss: 0.561253
Epoch: 2/4... Step: 700... Loss: 0.490149... Val Loss: 0.523221
Epoch: 2/4... Step: 800... Loss: 0.376533... Val Loss: 0.506046
Epoch: 3/4... Step: 900... Loss: 0.299006... Val Loss: 0.546762
Epoch: 3/4... Step: 1000... Loss: 0.308804... Val Loss: 0.455720
Epoch: 3/4... Step: 1100... Loss: 0.323383... Val Loss: 0.499651
Epoch: 3/4... Step: 1200... Loss: 0.345807... Val Loss: 0.469713
Epoch: 4/4... Step: 1300... Loss: 0.175339... Val Loss: 0.490351
Epoch: 4/4... Step: 1400... Loss: 0.252126... Val Loss: 0.443279
Epoch: 4/4... Step: 1500... Loss: 0.331545... Val Loss: 0.437071
Epoch: 4/4... Step: 1600... Loss: 

## Testing the trained model

In [22]:
# Get test data loss and accuracy

test_losses = []
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:
    
    h = tuple([each.data for each in h])
    
    if train_on_gpu:
        inputs, labels = inputs.cuda(), labels.cuda()
        
    output, h = net(inputs, h)
    
    #Calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze())
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    
# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.454
Test accuracy: 0.810


### Inference on a test review

In [24]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [25]:
def tokenize_review(test_review):
    test_review = test_review.lower()
    
    #Getting rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])
    
    #Splitting by spaces
    test_words = test_text.split(" ")
    
    #Tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])
    
    return test_ints

#Testing the above function
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 247, 18, 10, 28, 108, 113, 14, 388, 2, 10, 181, 60, 273, 144, 11, 18, 68, 76, 113, 2, 1, 410, 14, 539]]


In [26]:
#Test sequence padding
seq_length = 200
features = pad_features(test_ints, seq_length)
print(features)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   1 247  18  10  28
  108 113  14 388   2  10 181  60 273 144  11  18  68  76 113   2   1 410
   14 539]]


In [27]:
#Test conversion to tensor and passing into model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 200])


In [28]:
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    #Tokenize review
    test_ints = tokenize_review(test_review)
    
    #Pad tokenized sequence
    seq_length = sequence_length
    features = pad_features(test_ints, seq_length)
    
    #Conversion to tensor
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    #Initialize hidden state
    h = net.init_hidden(batch_size)
    
    if train_on_gpu:
        feature_tensor = feature_tensor.cuda()
        
    #Get output from the model
    output, h = net(feature_tensor, h)
    pred = torch.round(output.squeeze())
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    #Print custom response
    if pred.item() == 1:
        print("Positive review detected...")
    else:
        print("Negative review detected...")
        
    

In [29]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'

In [30]:
# call function
seq_length=200

predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.006548
Negative review detected...


In [31]:
# call function
seq_length=200

predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.990201
Positive review detected...


# END OF NOTEBOOK